In [12]:
import pandas as pd

In [2]:
import findspark
findspark.init()

In [21]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType, DoubleType

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('taxi_schema') \
    .getOrCreate()

24/03/02 20:23:33 WARN Utils: Your hostname, Muhammads-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.192 instead (on interface en0)
24/03/02 20:23:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 20:23:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
df_green_pd = pd.read_csv('data/raw/green/2021/01/green_tripdata_2021_01.csv.gz', nrows=100)

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                  int64
congestion_surcharge     float64
dtype: object

In [26]:
df_yellow_pd = pd.read_csv('data/raw/yellow/2021/01/yellow_tripdata_2021_01.csv.gz', nrows=100)

In [16]:
spark.createDataFrame(df_green_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [27]:
spark.createDataFrame(df_yellow_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', StringType(), True), StructField('tpep_dropoff_datetime', StringType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [23]:
green_schema = StructType([
    StructField('VendorID', IntegerType(), True), 
    StructField('lpep_pickup_datetime', TimestampType(), True), 
    StructField('lpep_dropoff_datetime', TimestampType(), True), 
    StructField('store_and_fwd_flag', StringType(), True), 
    StructField('RatecodeID', IntegerType(), True), 
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True), 
    StructField('passenger_count', IntegerType(), True), 
    StructField('trip_distance', DoubleType(), True), 
    StructField('fare_amount', DoubleType(), True), 
    StructField('extra', DoubleType(), True), 
    StructField('mta_tax', DoubleType(), True), 
    StructField('tip_amount', DoubleType(), True), 
    StructField('tolls_amount', DoubleType(), True), 
    StructField('ehail_fee', DoubleType(), True), 
    StructField('improvement_surcharge', DoubleType(), True), 
    StructField('total_amount', DoubleType(), True), 
    StructField('payment_type', IntegerType(), True), 
    StructField('trip_type', IntegerType(), True), 
    StructField('congestion_surcharge', DoubleType(), True)
])

In [32]:
year = 2020

for month in range(1, 13):
    print(f'Processing data for {year}/{month}')
    
    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'
    
    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)
    
    df_green \
    .repartition(4) \
    .write \
    .parquet(output_path)
    

Processing data for 2020/1


Processing data for 2020/2


Processing data for 2020/3


Processing data for 2020/4
Processing data for 2020/5
Processing data for 2020/6
Processing data for 2020/7
Processing data for 2020/8
Processing data for 2020/9
Processing data for 2020/10
Processing data for 2020/11
Processing data for 2020/12


In [28]:
yellow_schema = StructType([
    StructField('VendorID', IntegerType(), True), 
    StructField('tpep_pickup_datetime', TimestampType(), True), 
    StructField('tpep_dropoff_datetime', TimestampType(), True), 
    StructField('passenger_count', IntegerType(), True), 
    StructField('trip_distance', DoubleType(), True), 
    StructField('RatecodeID', IntegerType(), True), 
    StructField('store_and_fwd_flag', StringType(), True), 
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True), 
    StructField('payment_type', IntegerType(), True), 
    StructField('fare_amount', DoubleType(), True), 
    StructField('extra', DoubleType(), True), 
    StructField('mta_tax', DoubleType(), True), 
    StructField('tip_amount', DoubleType(), True), 
    StructField('tolls_amount', DoubleType(), True), 
    StructField('improvement_surcharge', DoubleType(), True), 
    StructField('total_amount', DoubleType(), True), 
    StructField('congestion_surcharge', DoubleType(), True)
])

In [34]:
year = 2020

for month in range(1, 13):
    print(f'Processing data for {year}/{month}')
    
    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'
    
    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)
    
    df_yellow \
    .repartition(4) \
    .write \
    .parquet(output_path)
    

Processing data for 2020/1


Processing data for 2020/2


Processing data for 2020/3


Processing data for 2020/4


Processing data for 2020/5


...

Processing data for 2020/8


Processing data for 2020/9


Processing data for 2020/10


Processing data for 2020/11


Processing data for 2020/12
